In [11]:
import sys
sys.path.append('..')

import os
import geopandas as gpd
import pandas as pd
from sqlalchemy import create_engine, text
import matplotlib.pyplot as plt

In [12]:
from config import RUTA_UNIDAD_ONE_DRIVE
from config import RUTA_LOCAL_ONE_DRIVE
from config import POSTGRES_UTEA

RUTA_COMPLETA = os.path.join(RUTA_UNIDAD_ONE_DRIVE, RUTA_LOCAL_ONE_DRIVE)
PATH_SHP_RECORRIDOS = RUTA_UNIDAD_ONE_DRIVE + r'\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Pulverizacion\2025\SHP\RECORRIDOS.shp'
GDF_RECORRIDOS = gpd.read_file(PATH_SHP_RECORRIDOS)

In [13]:
def obtener_engine():
    return create_engine(
        f"postgresql+psycopg2://{POSTGRES_UTEA['USER']}:{POSTGRES_UTEA['PASSWORD']}@{POSTGRES_UTEA['HOST']}:{POSTGRES_UTEA['PORT']}/{POSTGRES_UTEA['DATABASE']}"
    )

def obtener_parte_diario_sin_cod_labor():
    engine = obtener_engine()
    try:
        query = """
            SELECT DISTINCT idd, unidad_01, unidad_02
            FROM drones_pulverizacion.parte_diario_pulv 
            WHERE idd IS NOT NULL AND cod_labor IS NULL
        """
        # Usar pandas si no hay geometría involucrada
        df = pd.read_sql(query, engine)
        return df
    except Exception as e:
        print(f"❌ Error en la consulta: {e}")
        return pd.DataFrame()

def obtener_propiedad_por_idd(idd):
    engine = obtener_engine()
    try:
        query = f"""
            SELECT * FROM drones_pulverizacion.parte_diario_pulv WHERE idd= {idd}
        """
        gdf = gpd.read_postgis(query, engine, geom_col='geom')
        return gdf
    except Exception as e:
        print(f"❌ Error en la consulta: {e}")
        return gpd.GeoDataFrame()
    return None

def actualizar_area_rociada(id, val):
    engine = obtener_engine()
    try:
        with engine.begin() as conn:
            query = text("""
                UPDATE drones_pulverizacion.parte_diario_pulv
                SET 
                    area_rocia= :val
                WHERE id = :id
            """)
            conn.execute(query, {"id": id, "val": val})
            print(f"✔️ Area rociada actualizada de lote id: {id}")
    except Exception as e:
        print(f"❌ Error al actualizar: {e}")

def actualizar_faltantes_por_idd(idd, fecha_intermedia, fecha_min, fecha_max, numero_semana, num_vuelos, total_flight_time, total_caudal):
    engine = obtener_engine()
    try:
        with engine.begin() as conn:
            query = text("""
                UPDATE drones_pulverizacion.parte_diario_pulv
                SET 
                    fecha= :fecha_intermedia,
                    hora_ini= :fecha_min,
                    hora_fin= :fecha_max,
                    semana= :numero_semana,
                    num_vuelos= :num_vuelos,
                    horas_vuel= :total_flight_time,
                    total_caud= :total_caudal
                WHERE idd = :idd
            """)
            conn.execute(query, {
                "idd": idd, 
                "fecha_intermedia": fecha_intermedia,
                "fecha_min": fecha_min,
                "fecha_max": fecha_max,
                "numero_semana": numero_semana,
                "num_vuelos": num_vuelos,
                "total_flight_time": total_flight_time,
                "total_caudal": total_caudal,
            })
            print(f"✔️ Datos faltantes de idd: {idd} cargados correctamente")
    except Exception as e:
        print(f"❌ Error al actualizar: {e}")

In [14]:
propiedades_procesar = obtener_parte_diario_sin_cod_labor()
propiedades_procesar

,idd,unidad_01,unidad_02
0,84,664,SAN ANTONIO II--GUTIERREZ MARCIAL


In [15]:
selec_recorridos

,file,fecha,hora,id,drone,ctrl_id,pilot,fl_time,mode,height,spacing,fl_speed,area,spray,idd,geometry,fl_time_sec
1887,T50-1_20260114082426_R2772135228.kml,2026-01-14 00:00:00,08:24:26,R2772135228,T50-1,1581F6BUB24630011R08,MARIO SANCHEZ,9:40,Auto,4.0,8.0,28.080001,2.355333,23.846,80,"POLYGON ((487242.625 8084558.172, 487014.954 8...",580
1888,T50-1_20260114083655_R3392834185.kml,2026-01-14 00:00:00,08:36:55,R3392834185,T50-1,1581F6BUB24630011R08,MARIO SANCHEZ,8:43,Auto,4.0,8.0,28.080001,2.575333,25.983,80,"POLYGON ((487009.845 8084585.840, 487009.454 8...",523
1889,T50-1_20260114084826_R3893397860.kml,2026-01-14 00:00:00,08:48:26,R3893397860,T50-1,1581F6BUB24630011R08,MARIO SANCHEZ,9:51,Auto,4.0,8.0,28.080001,2.462000,24.991,80,"POLYGON ((487009.565 8084633.682, 487008.190 8...",591
1890,T50-1_20260114090132_R4494074270.kml,2026-01-14 00:00:00,09:01:32,R4494074270,T50-1,1581F6BUB24630011R08,MARIO SANCHEZ,9:23,Auto,4.0,8.0,28.080001,2.474000,24.992,80,"POLYGON ((487014.418 8084713.625, 487013.817 8...",563
1891,T50-1_20260114091342_R4814435022.kml,2026-01-14 00:00:00,09:13:42,R4814435022,T50-1,1581F6BUB24630011R08,MARIO SANCHEZ,10:55,Auto,4.0,8.0,28.080001,2.268667,23.096,80,"POLYGON ((487274.802 8084556.276, 487274.721 8...",655
1892,T50-1_20260114092933_R5294976150.kml,2026-01-14 00:00:00,09:29:33,R5294976150,T50-1,1581F6BUB24630011R08,MARIO SANCHEZ,3:41,Auto,4.0,8.0,28.080001,0.402000,4.195,80,"POLYGON ((487263.007 8084559.126, 487262.904 8...",221
1895,T50-1_20260114105546_R6436261329.kml,2026-01-14 00:00:00,10:55:46,R6436261329,T50-1,1581F6BUB24630011R08,MARIO SANCHEZ,9:41,Auto,4.0,8.0,28.080001,1.487333,15.224,80,"POLYGON ((486697.321 8085579.338, 486697.279 8...",581
1896,T50-1_20260114110839_R6876757363.kml,2026-01-14 00:00:00,11:08:39,R6876757363,T50-1,1581F6BUB24630011R08,MARIO SANCHEZ,10:23,Auto,4.0,8.0,28.080001,2.462000,24.986,80,"POLYGON ((486711.910 8085859.843, 486711.854 8...",623
1897,T50-1_20260114112217_R7237163209.kml,2026-01-14 00:00:00,11:22:17,R7237163209,T50-1,1581F6BUB24630011R08,MARIO SANCHEZ,10:02,Auto,4.0,8.0,28.080001,2.670000,26.984,80,"POLYGON ((486631.111 8085796.887, 486631.022 8...",602
1898,T50-1_20260114113912_R7457411226.kml,2026-01-14 00:00:00,11:39:12,R7457411226,T50-1,1581F6BUB24630011R08,MARIO SANCHEZ,11:10,Auto,4.0,8.0,28.080001,2.454000,24.984,80,"POLYGON ((486512.121 8085877.669, 486514.038 8...",670


In [16]:
for i, row in propiedades_procesar.iterrows():
    idd = row.idd
    selec_recorridos = GDF_RECORRIDOS[GDF_RECORRIDOS['idd']==idd]
    if (len(selec_recorridos) == 0):
        continue
    # selccion de lotes planificados
    selec_plan = obtener_propiedad_por_idd(idd)
    # conviente en campo fecha a formato fecha
    selec_recorridos.loc[:, "fecha"] = pd.to_datetime(selec_recorridos["fecha"])
    # Obtener la fecha mínima y máxima
    fecha_min = selec_recorridos["fecha"].min()
    fecha_max = selec_recorridos["fecha"].max()
    fecha_intermedia = fecha_min + (fecha_max - fecha_min) / 2
    print(fecha_max)
    numero_semana = fecha_intermedia.isocalendar().week
    area_total_pulv = selec_recorridos['area'].sum()
    num_vuelos = len(selec_recorridos)
    total_caudal = selec_recorridos['spray'].sum()
    
    selec_recorridos = selec_recorridos.copy()
    
    # obtener horas de vuelo en formto 00:00:00
    selec_recorridos["fl_time_sec"] = selec_recorridos["fl_time"].apply(lambda x: int(x.split(":")[0]) * 60 + int(x.split(":")[1]))
    # Sum all seconds
    total_seconds = selec_recorridos["fl_time_sec"].sum()
    # Convert back to hh:mm:ss format
    hours = total_seconds // 3600
    minutes = (total_seconds % 3600) // 60
    seconds = total_seconds % 60
    total_flight_time = f"{hours:02}:{minutes:02}:{seconds:02}"  # 00:00:00

    # Aplicar el buffer a la geometría de los recorridod
    selec_recorridos["geometry"] = selec_recorridos.geometry.buffer(5.5)
    # Disolver todas las geometrías en un solo polígono
    buffer_dissolved = selec_recorridos.dissolve()
    # Calcular la intersección
    interseccion = gpd.overlay(buffer_dissolved, selec_plan, how="intersection")
    # Calcular el área de cada polígono de la intersección
    interseccion["area_rociado"] = interseccion.geometry.area / 10000
    # seleccion solo de la columnas de interes
    resumen = interseccion[['id_2', 'unidad_05', 'area_2', 'area_rociado']].copy()
    
    # PROCEDIMIENTO PARA AJUSTAR EL AREA DE ROCIADO
    suma_area_rociada = resumen['area_rociado'].sum()
    area_rociada_real = area_total_pulv  # aquí pon tu valor real
    # Factor inicial de ajuste
    factor_ajuste = area_rociada_real / suma_area_rociada
    # Ajuste proporcional inicial
    resumen['area_ajustada'] = resumen['area_rociado'] * factor_ajuste
    # Verificar que no exceda area_2 y corregir iterativamente si es necesario
    while any(resumen['area_ajustada'] > resumen['area_2']):
        exceso = resumen['area_ajustada'] > resumen['area_2']
        resumen.loc[exceso, 'area_ajustada'] = resumen.loc[exceso, 'area_2']
        # Recalcular factor solo con las filas no ajustadas al máximo permitido
        area_ajustada_parcial = resumen.loc[~exceso, 'area_ajustada'].sum()
        area_restante = area_rociada_real - resumen.loc[exceso, 'area_ajustada'].sum()
        # Evitar división por cero
        if area_ajustada_parcial == 0:
            break
        nuevo_factor = area_restante / area_ajustada_parcial
        resumen.loc[~exceso, 'area_ajustada'] *= nuevo_factor
    # calcular campos faltantes
    resumen["porcen_rociado"] = (resumen["area_ajustada"]) / (resumen["area_2"])
    resumen["dif_area"] = resumen["area_2"] - resumen["area_ajustada"]
    # recorrer resumen para actualizar area rociada en el parte diario
    for i, row in resumen.iterrows():
        print(row.id_2, row.area_ajustada)
        actualizar_area_rociada(row.id_2, row.area_ajustada)
    # cargar campos restantes del parte con respecto a idd
    actualizar_faltantes_por_idd(idd, fecha_intermedia, fecha_min, fecha_max, numero_semana, num_vuelos, total_flight_time, total_caudal)

2026-01-14 00:00:00
735 0.8484846495643952
✔️ Area rociada actualizada de lote id: 735
737 1.396802826847468
✔️ Area rociada actualizada de lote id: 737
736 6.449379233728138
✔️ Area rociada actualizada de lote id: 736
✔️ Datos faltantes de idd: 84 cargados correctamente
